In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local[4]").setAppName("WindowFunction")

from pyspark.sql import SparkSession
# spark Session, entry point for Spark SQL, DataFrame
spark = SparkSession.builder\
                    .config(conf=config)\
                    .getOrCreate()

sc = spark.sparkContext

22/06/03 19:00:40 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.174.129 instead (on interface ens33)
22/06/03 19:00:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark-3.1.3-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-cdb0504e-39f3-448d-b753-1b251525ee21;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.3 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 1654ms :: artifacts dl 11ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from centra

In [3]:
data = [ ("James", "Sales", 3000),
    ("Michael", "Sales", 4600),
    ("Robert", "Sales", 4100),
    ("Maria", "Finance", 3000),
    ("James", "Sales", 3000),
    ("Scott", "Finance", 3300),
    ("Jen", "Finance", 3900),
    ("Jeff", "Marketing", 3000),
    ("Kumar", "Marketing", 2000),
    ("Saif", "Sales", 4100),
    ("Joe", "Sales", 4200),
    ("Venkat", "Sales", 4000),
        
   ]

empDf = spark.createDataFrame(data=data, schema=['name', 'dept', 'salary'])
empDf.printSchema()
empDf.show()

empDf.rdd.getNumPartitions()

root
 |-- name: string (nullable = true)
 |-- dept: string (nullable = true)
 |-- salary: long (nullable = true)



+-------+---------+------+
|   name|     dept|salary|
+-------+---------+------+
|  James|    Sales|  3000|
|Michael|    Sales|  4600|
| Robert|    Sales|  4100|
|  Maria|  Finance|  3000|
|  James|    Sales|  3000|
|  Scott|  Finance|  3300|
|    Jen|  Finance|  3900|
|   Jeff|Marketing|  3000|
|  Kumar|Marketing|  2000|
|   Saif|    Sales|  4100|
|    Joe|    Sales|  4200|
| Venkat|    Sales|  4000|
+-------+---------+------+



4

In [4]:
empDf.rdd.glom().collect()

[[Row(name='James', dept='Sales', salary=3000),
  Row(name='Michael', dept='Sales', salary=4600),
  Row(name='Robert', dept='Sales', salary=4100)],
 [Row(name='Maria', dept='Finance', salary=3000),
  Row(name='James', dept='Sales', salary=3000),
  Row(name='Scott', dept='Finance', salary=3300)],
 [Row(name='Jen', dept='Finance', salary=3900),
  Row(name='Jeff', dept='Marketing', salary=3000),
  Row(name='Kumar', dept='Marketing', salary=2000)],
 [Row(name='Saif', dept='Sales', salary=4100),
  Row(name='Joe', dept='Sales', salary=4200),
  Row(name='Venkat', dept='Sales', salary=4000)]]

In [4]:
empDf.createOrReplaceTempView("employees")

In [5]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# specification for window, partitions, functions that should be applied on partition
# with in department, order the data based on salary in ascending order
windowSpec = Window.partitionBy("dept").orderBy("salary")
# we have apply the spec on dataframe
df = empDf.withColumn("slno", row_number().over(windowSpec))

df.printSchema()
df.show()

df.filter (df.slno == 1).show()

root
 |-- name: string (nullable = true)
 |-- dept: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- slno: integer (nullable = true)



+-------+---------+------+----+
|   name|     dept|salary|slno|
+-------+---------+------+----+
|  James|    Sales|  3000|   1|
|  James|    Sales|  3000|   2|
| Venkat|    Sales|  4000|   3|
| Robert|    Sales|  4100|   4|
|   Saif|    Sales|  4100|   5|
|    Joe|    Sales|  4200|   6|
|Michael|    Sales|  4600|   7|
|  Maria|  Finance|  3000|   1|
|  Scott|  Finance|  3300|   2|
|    Jen|  Finance|  3900|   3|
|  Kumar|Marketing|  2000|   1|
|   Jeff|Marketing|  3000|   2|
+-------+---------+------+----+



+-----+---------+------+----+
| name|     dept|salary|slno|
+-----+---------+------+----+
|James|    Sales|  3000|   1|
|Maria|  Finance|  3000|   1|
|Kumar|Marketing|  2000|   1|
+-----+---------+------+----+



In [8]:
# sql

spark.sql("""
SELECT name, dept, salary, ROW_NUMBER() OVER (PARTITION BY dept ORDER BY SALARY) as slno FROM employees
""").show()

+-------+---------+------+----+
|   name|     dept|salary|slno|
+-------+---------+------+----+
|  James|    Sales|  3000|   1|
|  James|    Sales|  3000|   2|
| Venkat|    Sales|  4000|   3|
| Robert|    Sales|  4100|   4|
|   Saif|    Sales|  4100|   5|
|    Joe|    Sales|  4200|   6|
|Michael|    Sales|  4600|   7|
|  Maria|  Finance|  3000|   1|
|  Scott|  Finance|  3300|   2|
|    Jen|  Finance|  3900|   3|
|  Kumar|Marketing|  2000|   1|
|   Jeff|Marketing|  3000|   2|
+-------+---------+------+----+



In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

# rank with gap with ascending order
"""
score  rank
90      1
90      1
89      3  [gap, 2 not included]
"""
windowSpec = Window.partitionBy("dept").orderBy("salary")

df = empDf.withColumn("rank", rank().over(windowSpec))
df.show()

In [9]:
# sql

spark.sql("""
SELECT name, dept, salary, RANK() OVER (PARTITION BY dept ORDER BY SALARY) as rank FROM employees
""").show()

+-------+---------+------+----+
|   name|     dept|salary|rank|
+-------+---------+------+----+
|  James|    Sales|  3000|   1|
|  James|    Sales|  3000|   1|
| Venkat|    Sales|  4000|   3|
| Robert|    Sales|  4100|   4|
|   Saif|    Sales|  4100|   4|
|    Joe|    Sales|  4200|   6|
|Michael|    Sales|  4600|   7|
|  Maria|  Finance|  3000|   1|
|  Scott|  Finance|  3300|   2|
|    Jen|  Finance|  3900|   3|
|  Kumar|Marketing|  2000|   1|
|   Jeff|Marketing|  3000|   2|
+-------+---------+------+----+



In [17]:
from pyspark.sql.window import Window
from pyspark.sql.functions import first, last


windowSpec = Window.partitionBy("dept").orderBy("salary")\
                  .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)


df = empDf.withColumn("first", first("salary").over(windowSpec))\
          .withColumn("last", last("salary").over(windowSpec))
df.show()

+-------+---------+------+-----+----+
|   name|     dept|salary|first|last|
+-------+---------+------+-----+----+
|  James|    Sales|  3000| 3000|4600|
|  James|    Sales|  3000| 3000|4600|
| Venkat|    Sales|  4000| 3000|4600|
| Robert|    Sales|  4100| 3000|4600|
|   Saif|    Sales|  4100| 3000|4600|
|    Joe|    Sales|  4200| 3000|4600|
|Michael|    Sales|  4600| 3000|4600|
|  Maria|  Finance|  3000| 3000|3900|
|  Scott|  Finance|  3300| 3000|3900|
|    Jen|  Finance|  3900| 3000|3900|
|  Kumar|Marketing|  2000| 2000|3000|
|   Jeff|Marketing|  3000| 2000|3000|
+-------+---------+------+-----+----+



In [20]:
# sql

spark.sql("""
SELECT name, dept, salary, 

        FIRST_VALUE(salary) OVER (PARTITION BY dept ORDER BY SALARY 
                                RANGE BETWEEN UNBOUNDED PRECEDING AND 
                                UNBOUNDED FOLLOWING) as first ,
                                
        LAST_VALUE(salary) OVER (PARTITION BY dept ORDER BY SALARY 
                                RANGE BETWEEN UNBOUNDED PRECEDING AND 
                                UNBOUNDED FOLLOWING) as last 
                                
                                FROM employees
""").show()

+-------+---------+------+-----+----+
|   name|     dept|salary|first|last|
+-------+---------+------+-----+----+
|  James|    Sales|  3000| 3000|4600|
|  James|    Sales|  3000| 3000|4600|
| Venkat|    Sales|  4000| 3000|4600|
| Robert|    Sales|  4100| 3000|4600|
|   Saif|    Sales|  4100| 3000|4600|
|    Joe|    Sales|  4200| 3000|4600|
|Michael|    Sales|  4600| 3000|4600|
|  Maria|  Finance|  3000| 3000|3900|
|  Scott|  Finance|  3300| 3000|3900|
|    Jen|  Finance|  3900| 3000|3900|
|  Kumar|Marketing|  2000| 2000|3000|
|   Jeff|Marketing|  3000| 2000|3000|
+-------+---------+------+-----+----+



In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc

# rank with gap
"""
score  rank
90      1
90      1
89      3  [gap, 2 not included]
"""
windowSpec = Window.partitionBy("dept").orderBy(desc("salary"))

df = empDf.withColumn("rank", rank().over(windowSpec))
df.show()

In [21]:
# sql

spark.sql("""
SELECT name, dept, salary, RANK() OVER (PARTITION BY dept ORDER BY SALARY DESC) as rank FROM employees
""").show()

+-------+---------+------+----+
|   name|     dept|salary|rank|
+-------+---------+------+----+
|Michael|    Sales|  4600|   1|
|    Joe|    Sales|  4200|   2|
| Robert|    Sales|  4100|   3|
|   Saif|    Sales|  4100|   3|
| Venkat|    Sales|  4000|   5|
|  James|    Sales|  3000|   6|
|  James|    Sales|  3000|   6|
|    Jen|  Finance|  3900|   1|
|  Scott|  Finance|  3300|   2|
|  Maria|  Finance|  3000|   3|
|   Jeff|Marketing|  3000|   1|
|  Kumar|Marketing|  2000|   2|
+-------+---------+------+----+



In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank, desc

# dense_rank ranking without gap
"""
score  rank
90      1
90      1
89      2  
"""
windowSpec = Window.partitionBy("dept").orderBy(desc("salary"))

df = empDf.withColumn("rank", dense_rank().over(windowSpec))
df.show()

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import percent_rank, desc

# percent_rank ranking with perecent calculation
"""
 
"""
windowSpec = Window.partitionBy("dept").orderBy(desc("salary"))

df = empDf.withColumn("rank", percent_rank().over(windowSpec))
df.show()

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import ntile, desc

# ntile ranking with related certain range for range
# rank shall fit into a range
"""
 
"""
windowSpec = Window.partitionBy("dept").orderBy(desc("salary"))

df = empDf.withColumn("rank", ntile(4).over(windowSpec))
df.show()

In [ ]:
# Analytic functions
# Cumulative distribution - similar to rank, calcualted and values are bound between 
# 0 and 1

# 10 USD per share => 13 USD per share      = 3 USD per share, 30 % gain .3
# 100 USD per share => 110 USD per share    = 10 USD per share, 10% gain .1
# cumulative distribution
from pyspark.sql.window import Window
from pyspark.sql.functions import cume_dist, desc

# similar to  rank  
 
windowSpec = Window.partitionBy("dept").orderBy(desc("salary"))

df = empDf.withColumn("cume_dist", cume_dist().over(windowSpec))
df.show()

In [ ]:
# lag - previous lag
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, desc

windowSpec = Window.partitionBy("dept").orderBy("salary")

df = empDf.withColumn("lag", lag("salary",1).over(windowSpec))
df.show()

In [ ]:
# lead -  the one ahead, 
from pyspark.sql.window import Window
from pyspark.sql.functions import lead, desc

windowSpec = Window.partitionBy("dept").orderBy("salary")

df = empDf.withColumn("lead", lead("salary", 1).over(windowSpec))
df.show()

In [ ]:
# aggregate functions, min, max, sum, count, avg

from pyspark.sql.window import Window
from pyspark.sql.functions import avg, sum, min, max, count, col

windowSpec = Window.partitionBy("dept")

df = empDf\
          .withColumn("min", min(col("salary")).over(windowSpec))\
          .withColumn("max", max(col("salary")).over(windowSpec))\
          .withColumn("avg", avg(col("salary")).over(windowSpec))\
          .withColumn("count", count(col("salary")).over(windowSpec))\
          .withColumn("sum", sum(col("salary")).over(windowSpec))

df.show()

In [32]:
# CTE - Common Table expression , create/define a temp table, which can be referenced later 1 or more times
# with partition
spark.sql("""
WITH salaries AS 
(SELECT *, RANK() OVER (PARTITION BY dept ORDER BY salary) as rank from employees)
SELECT * FROM salaries WHERE rank = 1
""").show()

+-----+---------+------+----+
| name|     dept|salary|rank|
+-----+---------+------+----+
|James|    Sales|  3000|   1|
|James|    Sales|  3000|   1|
|Maria|  Finance|  3000|   1|
|Kumar|Marketing|  2000|   1|
+-----+---------+------+----+



In [33]:
# CTE - Common Table expression , create/define a temp table, which can be referenced later 1 or more times
# scope of CTE is single sql statement
# with partition
spark.sql("""
WITH salaries AS 
(SELECT *, RANK() OVER (ORDER BY salary) as rank from employees)
SELECT * FROM salaries WHERE rank = 1
""").show()

22/06/04 01:46:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----+---------+------+----+
| name|     dept|salary|rank|
+-----+---------+------+----+
|Kumar|Marketing|  2000|   1|
+-----+---------+------+----+



In [34]:
# CTE - Common Table expression , create/define a temp table, which can be referenced later 1 or more times
# with partition
spark.sql("""
WITH salaries AS 
(SELECT *, ROW_NUMBER() OVER (ORDER BY salary) as row from employees)
SELECT * FROM salaries WHERE row = 2
""").show()

22/06/04 01:47:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----+-----+------+---+
| name| dept|salary|row|
+-----+-----+------+---+
|James|Sales|  3000|  2|
+-----+-----+------+---+



In [35]:
spark.sql("""
CREATE TEMPORARY VIEW number1(c) AS VALUES (3), (1), (2), (2), (3), (4);
""")

DataFrame[]

In [36]:
spark.sql("SELECT c FROM number1").show()

+---+
|  c|
+---+
|  3|
|  1|
|  2|
|  2|
|  3|
|  4|
+---+



In [40]:
spark.sql("""
 
 SELECT  distinct salary from employees
 
""").show()

+------+
|salary|
+------+
|  3300|
|  4000|
|  4200|
|  3900|
|  4100|
|  4600|
|  3000|
|  2000|
+------+



In [48]:
spark.sql("""
WITH s AS
(
WITH salaries AS 
 (SELECT  distinct salary from employees)
SELECT *, ROW_NUMBER() OVER (ORDER BY salary) as row  FROM salaries
)
select s.* from s WHERE row=2
""").show()

22/06/04 02:55:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------+---+
|salary|row|
+------+---+
|  3000|  2|
+------+---+

